In [1]:
import pandas as pd 
import numpy as np
import functools
import os


## Data Wrangling

### Load Data

In [2]:
data_path = "../data/"

ghg_mtco2 = pd.read_csv(os.path.join(data_path, "ghg_mtco2.csv"))
ghg_pergdp = pd.read_csv(os.path.join(data_path, "ghg_pergdp.csv"))
gdp_percapita = pd.read_csv(os.path.join(data_path, "gdp.csv"), 
                            header=2)
population = pd.read_csv(os.path.join(data_path, "population.csv"), 
                         header=2)
per_capita_energysource = pd.read_csv(os.path.join(data_path, "per_capita_energysource.csv"))
cumulative_prod = pd.read_csv(os.path.join(data_path, "cumulative_prod.csv")) 

### Extract Countries of Interest

In [3]:
# init country list
countries_of_interest = ["Canada", "Japan", "France", "Germany", "Italy", "United States", "United Kingdom", # G7 
                         "China", "India", "Indonesia", "Brazil", "Iran", "Mexico", "South Africa"] # Dev 7

# ensure country names are consistent
gdp_percapita = gdp_percapita.replace(to_replace="Iran, Islamic Rep.", value="Iran")
population = population.replace(to_replace="Iran, Islamic Rep.", value="Iran")

# extract
ghg_mtco2 = ghg_mtco2.loc[ghg_mtco2["Country/Region"].isin(countries_of_interest)]
ghg_pergdp = ghg_pergdp.loc[ghg_pergdp["Country/Region"].isin(countries_of_interest)]
gdp_percapita = gdp_percapita.loc[gdp_percapita["Country Name"].isin(countries_of_interest)]
population = population.loc[population["Country Name"].isin(countries_of_interest)]
per_capita_energysource = per_capita_energysource.loc[per_capita_energysource["Entity"].isin(countries_of_interest)]
cumulative_prod = cumulative_prod.loc[cumulative_prod["country"].isin(countries_of_interest)]

### Drop Unnecessary Columns and Dates

We are only concerned with values ranging from 1990 to 2018.

In [4]:
# drop unnecessary features
ghg_mtco2 = ghg_mtco2.drop(columns=["unit"])
ghg_pergdp = ghg_pergdp.drop(columns=["unit"])
gdp_percapita = gdp_percapita.drop(columns=["Country Code", "Indicator Name", 
                                            "Indicator Code", "Unnamed: 65"])
population = population.drop(columns=["Country Code", "Indicator Name", 
                                            "Indicator Code", "Unnamed: 65"])
per_capita_energysource = per_capita_energysource.drop(columns=["Code"])
cumulative_prod = cumulative_prod.loc[:, ["country", "year", 
                                          "cumulative_oil_co2", "cumulative_gas_co2",
                                          "cumulative_flaring_co2", "cumulative_coal_co2",
                                          "cumulative_cement_co2"]]


In [5]:
# Since the impetus for this project is Fixed Effects regression
# We want our data to be annual independent of its previous sample
# Hence, we convert cumulative data to yearly

def cumulative_to_annual(data):
    return data.transform(lambda sample: sample.sub(sample.shift().fillna(0)))

cumulative_prod.iloc[:, 2:] = cumulative_to_annual(cumulative_prod.iloc[:, 2:])

In [6]:
# drop dates outside of scope i.e. not 1990-2018

# columns
x_dates = np.append(np.arange(1960, 1990), np.array([2019, 2020])).flatten()
x_dates = [str(date) for date in x_dates]

gdp_percapita = gdp_percapita.drop(columns=x_dates)
population = population.drop(columns=x_dates)

# indices
pces_idx = per_capita_energysource[(per_capita_energysource["Year"]<1990) | (per_capita_energysource["Year"]>2018)].index
cp_idx = cumulative_prod[(cumulative_prod["year"]<1990) | (cumulative_prod["year"]>2018)].index

per_capita_energysource = per_capita_energysource.drop(index=pces_idx)
cumulative_prod = cumulative_prod.drop(index=cp_idx)

In [7]:
# for some reason there are "false" string values in the encoded data for Iran
# we replace them with NaNs 
ghg_pergdp = ghg_pergdp.replace("false", np.nan)

### Format/ensure countries and dates into columns

The most crucial part. Most of the datasets uses the dates as columns, and countries as samples. We have to convert it to **tidy** format such that each column is a variable and each row is an observation. We also use the country and year as index, so as to follow the panel data format.

In [8]:
def stack_reset_and_rename(df, index_col=None, feature_name=None, to_format=True, to_rename=True):
    if to_format:
        df = df.set_index(index_col)
        df = df.stack().reset_index()
        
    if to_rename:
        df.columns = ["Country", "Year", feature_name]
    
    # ensure year has consistent data types
    # to avoid errors in merging 
    df["Year"] = pd.to_datetime(df["Year"], format="%Y")
    df["Year"] = pd.DatetimeIndex(df["Year"]).year
    # sort values by country and year
    df = df.sort_values(by=["Country", "Year"])
    df = df.set_index(["Country", "Year"])
    # cast numerical values to float 32
    df = df.astype(dtype=np.float32)
    return df

# ensure same country and year column names
# fo simplify queries in the stack_reset_and_rename func
per_capita_energysource = per_capita_energysource.rename(columns={"Entity": "Country"})
cumulative_prod = cumulative_prod.rename(columns={"country": "Country", 
                                                  "year": "Year"})

# format
ghg_mtco2 = stack_reset_and_rename(ghg_mtco2, "Country/Region", "ghg_mtco2")
ghg_pergdp = stack_reset_and_rename(ghg_pergdp, "Country/Region", "ghg_pergdp")
gdp_percapita = stack_reset_and_rename(gdp_percapita, "Country Name", "gdp_percapita")
population = stack_reset_and_rename(population, "Country Name", "population")
# per_capita_energysource has more than 1 column we want to rename
# we'll just do it manually
per_capita_energysource = stack_reset_and_rename(per_capita_energysource, 
                                                 to_rename=False,
                                                 to_format=False)
per_capita_energysource.columns = ["fossilfuels_percapita", "nuclear_percapita",
                                   "renewables_percapita"]
# Also, due to our changing of the cumulative features to annual
# we also have to rename the columns in cumulative prod
cumulative_prod = stack_reset_and_rename(cumulative_prod, to_format=False, 
                                         to_rename=False)
cumulative_prod.columns = ["yearly_oil_co2", "yearly_gas_co2", "yearly_flaring_co2", 
                           "yearly_coal_co2", "yearly_cement_co2"]

### Unify to one big dataset

In [9]:
data_frames = [ghg_mtco2, ghg_pergdp, gdp_percapita, population, per_capita_energysource, cumulative_prod]

model_data = functools.reduce(lambda  left,right: pd.merge(left,right,on=["Country", "Year"],
                                                           how='outer').astype(np.float32), data_frames)

# save data for later use
model_data.to_csv(os.path.join(data_path, "model_data.csv"))

In [10]:
%load_ext watermark
%watermark -v -iv 

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.28.0

pandas: 1.3.4
numpy : 1.20.3
sys   : 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:15:42) [MSC v.1916 64 bit (AMD64)]

